# OM 621 — Advanced Visual Analytics  
## Assignments 2 & 3 — Transportation Cost Estimation & Forecasting (Combined)

**Student:** Brenda Laime Jalil  
**Term:** Fall 2025  

> Single notebook that **merges A2 + A3**. I keep it simple and readable focusing on two things:  
> 1) **Invoice delay** (when invoices arrive vs when we ship), and  
> 2) **Invoice time‑series by mode** (seasonality & trend).

### How to Run
1. Keep the dataset at `../data/tr_data_22_24.csv` (already included here).  
2. Run cells top‑to‑bottom.  
3. Figures will be saved into `../plots/`.

In [ ]:

import os, pandas as pd, numpy as np
from plotnine import (ggplot, aes, geom_col, geom_line, geom_point, geom_boxplot, 
                      geom_histogram, geom_density, labs, facet_wrap, coord_flip, 
                      theme_minimal, theme, element_text)
from plotnine.themes import theme_tufte

pd.set_option("display.max_columns", 100)

PLOTS_DIR = "../plots"
os.makedirs(PLOTS_DIR, exist_ok=True)
print("Saving figures to:", PLOTS_DIR)


## Load Data
I load the single CSV and take a quick peek.

In [ ]:

csv_path = "../data/tr_data_22_24.csv"
df = pd.read_csv(csv_path)
df.head()


## Clean + Derived Columns
- Parse dates and build **`delay`** in days.  
- Fix the known mode typo (`parcel_grund` → `parcel_ground`).  
- Create a **readable `mode_label`** for charts.

In [ ]:

df["shipping_date"] = pd.to_datetime(df["shipping_date"], errors="coerce")
df["invoice_date"]  = pd.to_datetime(df["invoice_date"],  errors="coerce")
df["delay"] = (df["invoice_date"] - df["shipping_date"]).dt.days

df["mode"] = df["mode"].astype("object").str.strip().replace({"parcel_grund":"parcel_ground"})

def nice_mode(m):
    if pd.isna(m): return m
    s = str(m).strip().lower()
    mapping = {
        "exp_air":"Expedited Air","std_air":"Standard Air",
        "full_container_load":"Full Container Load (FCL)","less_container_load":"Less Than Container Load (LCL)",
        "truck_load":"Full Truck Load (FTL)","less_truck_load":"Less Than Truck Load (LTL)",
        "parcel_ground":"Parcel Ground","parcel_air":"Parcel Air","other":"Other"
    }
    return mapping.get(s, s.replace("_"," ").title())

df["mode_label"] = df["mode"].apply(nice_mode)
print("Rows:", len(df))


---
# Assignment 2 
This section covers A2.Q1–Q5: expectations vs actual, basic exploration, first visuals, and the **delay** feature.

### A2.Q1 — Data Expectation (from my A1 plan)
**Did we get what I expected?** Yes. I expected at least: `shipping_date`, `invoice_date`, `usda_invoice_amount`, `mode`, `region`, `destination`, and `site`.  
They are present (with some missing in `site` or `division`).

**Challenges I anticipated & how I handle them here:**
- **Missing values** (esp. `site`/`division`): I always show effective sample sizes when I facet by those fields.  
- **Category cleanup** (`mode` typos): I cleaned the known one, and I trim/standardize strings.  
- **Dates & `delay`**: I convert to datetime with `errors="coerce"`, drop negatives for analysis, and summarize with medians/IQRs (right‑skewed tails are common).  
- **Skewed amounts**: I report median/IQR in addition to mean.  
- **Possible duplicates**: I do quick checks when grouping; not a blocker for these visuals.  
- **Scope gap**: No carrier terms or accessorials in this phase—fine for descriptive patterns, but I call that out in conclusions.

### A2.Q2 — Basic Exploration
Non‑null counts and simple invoice summaries.

In [ ]:

# Non‑null status
n = len(df); nn = df.notna().sum()
summary = (pd.DataFrame({"non_null": nn, "null": n-nn, "pct_complete": (nn/n*100).round(2)})
           .sort_values("pct_complete", ascending=False))
summary


In [ ]:

# Invoice stats
df["usda_invoice_amount"] = pd.to_numeric(df["usda_invoice_amount"], errors="coerce")
df["usda_invoice_amount"].agg(mean="mean", median="median",
    iqr=lambda s: s.quantile(0.75)-s.quantile(0.25), min="min", max="max").round(2)


**Quick read:** amounts are **right‑skewed** (a few very large values), so I lean on **median + IQR** for center/spread and keep the mean for context.

### A2.Q3 — Basic Visuals
**Q3(a)** Which site runs the most tasks?  
**Q3(b)** Does that change by region?  
**Q3(c)** Which transportation mode is used most?

In [ ]:

# Q3(a) — Tasks by Site (sorted; horizontal to keep labels clean)
site_counts = (df.groupby("site", dropna=False).size()
               .reset_index(name="n_tasks")
               .sort_values("n_tasks", ascending=True))  # ascending for coord_flip
top_site = site_counts[site_counts["site"].notna()].iloc[-1]["site"]
print("Top site:", top_site)

p_site = (ggplot(site_counts, aes("site","n_tasks"))
          + geom_col()
          + coord_flip()
          + labs(title="Transportation Tasks by Site", x="Site", y="Tasks")
          + theme_minimal())
p_site


In [ ]:

# Q3(b) — Tasks by Site within Region
site_region_counts=(df[df["site"].notna()]
                    .groupby(["region","site"], observed=True).size()
                    .reset_index(name="n_tasks"))

p_site_region = (ggplot(site_region_counts, aes("site","n_tasks"))
                 + geom_col()
                 + coord_flip()
                 + facet_wrap("~region", scales="free_y")
                 + labs(title="Tasks by Site — by Region", x="Site", y="Tasks")
                 + theme_minimal())
p_site_region


In [ ]:

# Q3(c) — Tasks by Mode (clean labels)
mode_counts=(df.groupby("mode_label", dropna=False, observed=True).size()
             .reset_index(name="n_tasks")
             .sort_values("n_tasks", ascending=True))

top_mode = mode_counts[mode_counts["mode_label"].notna()].iloc[-1]["mode_label"]
print("Top mode:", top_mode)

p_mode = (ggplot(mode_counts, aes("mode_label","n_tasks"))
          + geom_col()
          + coord_flip()
          + labs(title="Transportation Tasks by Mode", x="Mode", y="Tasks")
          + theme_minimal())
p_mode


### A2.Q4 — Delay Feature
I compute **delay** in days and look at it by **region** and **site** using histograms and densities.

In [ ]:

valid_delay = df[(df["delay"].notna()) & (df["delay"] >= 0)].copy()

# (a) Region — histogram
p_hist_region = (ggplot(valid_delay, aes("delay"))
                 + geom_histogram(bins=40)
                 + facet_wrap("~region", scales="free_y")
                 + labs(title="Invoice Delay — Histogram by Region", x="Delay (days)", y="Count")
                 + theme_minimal())
p_hist_region


In [ ]:

# (a) Region — density
p_den_region = (ggplot(valid_delay, aes("delay"))
                + geom_density()
                + facet_wrap("~region", scales="free_y")
                + labs(title="Invoice Delay — Density by Region", x="Delay (days)", y="Density")
                + theme_minimal())
p_den_region


In [ ]:

# (b) Top sites — histogram & density
top_sites = df["site"].value_counts(dropna=False).head(12).index
df_top = valid_delay[valid_delay["site"].isin(top_sites)].copy()

p_hist_site = (ggplot(df_top, aes("delay"))
               + geom_histogram(bins=40)
               + facet_wrap("~site", scales="free_y")
               + labs(title="Invoice Delay — Histogram by Top Sites", x="Delay (days)", y="Count")
               + theme_minimal())
p_hist_site


In [ ]:

p_den_site = (ggplot(df_top, aes("delay"))
              + geom_density()
              + facet_wrap("~site", scales="free_y")
              + labs(title="Invoice Delay — Density by Top Sites", x="Delay (days)", y="Density")
              + theme_minimal())
p_den_site


**What I see:** delays are **right‑skewed** in all regions/sites. Most invoices arrive within ~0–30 days, with occasional long tails.

#### A2.Q4(c) — Delay vs Invoice Amount (overall, by region, by top sites)
I check if bigger invoices tend to have longer delays. Short answer: not really, it's mostly a noisy cloud.

In [ ]:

df_scatter = df[(df["delay"].notna()) & (df["delay"] >= 0) & (df["usda_invoice_amount"].notna())].copy()

xmax = df_scatter["delay"].quantile(0.99)
ymax = df_scatter["usda_invoice_amount"].quantile(0.99)

p_scatter = (ggplot(df_scatter, aes("delay","usda_invoice_amount"))
             + geom_point(alpha=0.12)
             + labs(title="Delay vs Invoice Amount — Overall (zoomed at 99th pct)",
                    x="Delay (days)", y="Invoice Amount (USD)")
             + theme_minimal())
p_scatter


In [ ]:

p_scatter_region = (ggplot(df_scatter, aes("delay","usda_invoice_amount"))
                    + geom_point(alpha=0.12)
                    + facet_wrap("~region", scales="free_y")
                    + labs(title="Delay vs Invoice Amount — by Region",
                           x="Delay (days)", y="Invoice Amount (USD)")
                    + theme_minimal())
p_scatter_region


In [ ]:

top_sites_sc = df["site"].value_counts(dropna=False).head(6).index
df_scatter_sites = df_scatter[df_scatter["site"].isin(top_sites_sc)].copy()
p_scatter_site = (ggplot(df_scatter_sites, aes("delay","usda_invoice_amount"))
                  + geom_point(alpha=0.12)
                  + facet_wrap("~site", scales="free_y")
                  + labs(title="Delay vs Invoice Amount — Top Sites",
                         x="Delay (days)", y="Invoice Amount (USD)")
                  + theme_minimal())
p_scatter_site


**Takeaway:** I don’t see a strong relationship between delay and invoice size. Differences look driven by **mode/route mix**, not by delay itself.

### A2.Q5 — Delay by Mode (distribution)
Since the prompt asks for the **distribution**, I show **densities by mode** (and I exclude `Other`). This complements the boxplot in A3.

In [ ]:

valid_mode = valid_delay[df["mode"].notna()].copy()
valid_mode = valid_mode[valid_mode["mode"].str.lower() != "other"]

p_den_mode = (ggplot(valid_mode, aes("delay"))
              + geom_density()
              + facet_wrap("~mode_label", scales="free_y")
              + labs(title="Invoice Delay — Density by Mode", x="Delay (days)", y="Density")
              + theme_minimal())
p_den_mode


---
# Assignment 3
Now I focus on two deliverables:  
1) **Delay distribution by mode** with a **boxplot ordered by median** and  
2) **Invoice time series** that actually helps us see patterns (seasonality/trend) by **mode**.

## A3.Q1 — Delay Distribution by Mode (Ordered Boxplot)
I sort by **median delay**, flip the axes so long labels read clean, and save the PNG.

In [ ]:

df_q1 = valid_mode.copy()

order_by_median = (df_q1.groupby("mode_label")["delay"]
                   .median().sort_values(ascending=False).index)
df_q1["mode_label"] = pd.Categorical(df_q1["mode_label"], categories=order_by_median, ordered=True)

p_q1 = (ggplot(df_q1, aes("mode_label","delay"))
        + geom_boxplot()
        + coord_flip()
        + labs(title="Invoice Delay by Transportation Mode (Ordered by Median)",
               x="Mode", y="Delay (days)")
        + theme_tufte())
p_q1
p_q1.save(filename=f"{PLOTS_DIR}/delay_dist_by_mode.png", width=10, height=6, dpi=300)
print("Saved:", f"{PLOTS_DIR}/delay_dist_by_mode.png")


## A3.Q2 — Invoice Time Series
**First try:** a single daily total mixes modes and looks noisy (not good for decisions).  
**Better:** use **monthly totals**, draw **lines by year**, and **facet by mode**. That makes seasonality vs. level shifts obvious.

In [ ]:

# Initial view (noisy on purpose)
df_ts_daily = (df[df["shipping_date"].notna() & df["usda_invoice_amount"].notna()]
               .groupby("shipping_date", observed=True)["usda_invoice_amount"]
               .sum().reset_index(name="total_invoice_amount"))
p_ts_init = (ggplot(df_ts_daily, aes("shipping_date","total_invoice_amount"))
             + geom_line()
             + labs(title="Initial: Total Daily Invoice Amount (All Modes) — Very Noisy",
                    x="Shipping Date", y="Total Daily Invoice Amount (USD)")
             + theme_tufte())
p_ts_init


In [ ]:

# Better view: lines by year, faceted by mode
df_modes = df[df["mode"].notna()].copy()
df_modes = df_modes[df_modes["mode"].str.lower() != "other"]
df_modes["mode_label"] = df_modes["mode"].apply(nice_mode)
df_modes["year"]  = df_modes["shipping_date"].dt.year
df_modes["month"] = df_modes["shipping_date"].dt.month

monthly_mode = (df_modes.groupby(["year","month","mode_label"], observed=True)["usda_invoice_amount"]
                .sum().reset_index(name="monthly_invoice_amount"))

p_ts_mode = (ggplot(monthly_mode, aes("month","monthly_invoice_amount", group="year"))
             + geom_line()
             + facet_wrap("~mode_label", scales="free_y")
             + labs(title="Monthly Invoice Amount by Mode — Lines Grouped by Year",
                    x="Month (1–12)", y="Monthly Invoice Amount (USD)")
             + theme_tufte())
p_ts_mode
p_ts_mode.save(filename=f"{PLOTS_DIR}/invoice_ts_by_mode.png", width=14, height=8, dpi=300)
print("Saved:", f"{PLOTS_DIR}/invoice_ts_by_mode.png")


## A3.Q3 — Cost Estimation & Forecasting 
**Patterns I see:**  
- **Delay:** Container modes (LCL/FCL) have the **longest typical delays** and **fatter right tails**; parcel/air is the fastest.  
- **Invoices over time:** Clear **seasonality by month** for several modes. Container modes also show a **rising trend**; parcel looks fairly **flat/stable**.

**How I’d estimate/forecast:**  
- **Model by mode** (separately) to avoid mixing different behaviors.  
- For each mode, use a **seasonal monthly baseline** (last 2–3 years of the same month) and weight the **recent 3–6 months** to capture trend.  
- For accruals, time the spend with each mode’s **median delay** and add a **buffer** for container modes because of their long‑tail risk.  
- Sum the per‑mode forecasts for a total estimate. Keep monitoring outliers and update monthly.

This keeps the story actionable for the Director of Supply Chain: steady parcel costs vs. rising container costs means we protect budget for those container swings.

---
## References
- Karimi, M. (2025). *Grammar of Graphics; Visualization Refinement; Advancing Simple Graphics* [Class materials]. OM 621, CSUSM.  
- McKinney, W. (2017). *Python for Data Analysis* (2nd ed.). O’Reilly.    
- Course dataset: `tr_data_22_24.csv`.